In [190]:
from pathlib import Path

import numpy as np
import xarray as xr
from datatree import DataTree, open_datatree
from xarray import open_zarr

## Data Creation


In [191]:
xda = xr.DataArray(
    np.arange(3 * 18).reshape(3, 18),
    coords={"label": list("abc"), "z": list(range(18))},
)
xda = xda.chunk({"label": 2, "z": 4})
xda

<xarray.DataArray (label: 3, z: 18)>
dask.array<xarray-<this-array>, shape=(3, 18), dtype=int64, chunksize=(2, 4), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

In [192]:
xdt = DataTree(xr.Dataset({"my_xda": xda}))
xdt.my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<xarray-<this-array>, shape=(3, 18), dtype=int64, chunksize=(2, 4), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

## Data Writing


In [193]:
zarr_path = Path() / "../generated/my_array.zarr"
xdt.to_zarr(zarr_path)

## Data Reading


### Using xarray's `open_zarr` 

See https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html

Documentation version: `stable` at time of writing: 11 nov 2023

Resulting behaviours serve as reference.

#### No `chunks` kwarg

Stored chunks are used.

In [194]:
open_zarr(zarr_path).my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<open_dataset-my_xda, shape=(3, 18), dtype=int64, chunksize=(2, 4), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

#### With `chunks='auto'`

Stored chunks are used.

Same behaviour as with no `chunks` kwarg.

In [195]:
# https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html
open_zarr(zarr_path, chunks="auto").my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<open_dataset-my_xda, shape=(3, 18), dtype=int64, chunksize=(2, 4), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

### Using datatree's `open_datatree` with `engine='zarr'`

See https://xarray-datatree.readthedocs.io/en/latest/generated/datatree.open_datatree.html#datatree.open_datatree

Documentation version:

```
Datatree 0.0.14.dev5+g433f78d.d20231110 documentation
```

#### No `chunks` kwarg

No chunking performed.

(NOK)

(!) Differs from the xarray's reference behaviour where stored chunks are used.

In [196]:
open_datatree(zarr_path, engine="zarr").my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
[54 values with dtype=int64]
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

#### With `chunks='auto'`

A chunk identical to the shape of the data is used.
This means chunking is useless as there is only a single chunk representing the whole dataset

(NOK)

(!) Differs from the xarray's reference behaviour where stored chunks are used.


In [197]:
open_datatree(zarr_path, engine="zarr", chunks="auto").my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<open_dataset-my_xda, shape=(3, 18), dtype=int64, chunksize=(3, 18), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

#### With `chunks` kwarg (same as stored chunks)

(OK)

No warning is shown because given chunks correspond to the stored chunks

In [198]:
open_datatree(zarr_path, engine="zarr", chunks={"label": 2, "z": 4}).my_xda

<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<open_dataset-my_xda, shape=(3, 18), dtype=int64, chunksize=(2, 4), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

#### With `chunks` kwarg (differing from the stored chunks)

(OK)

A warning is shown

> UserWarning: The specified chunks separate the stored chunks along dimension "z" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.

In [199]:
open_datatree(zarr_path, engine="zarr", chunks={"label": 999, "z": 5}).my_xda

/home/tselano/.cache/pypoetry/virtualenvs/datatree-experimentation-Sa4oWCLA-py3.10/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "z" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.DataArray 'my_xda' (label: 3, z: 18)>
dask.array<open_dataset-my_xda, shape=(3, 18), dtype=int64, chunksize=(3, 5), chunktype=numpy.ndarray>
Coordinates:
  * label    (label) <U1 'a' 'b' 'c'
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17